In [1]:
import numpy as np 
import os
from glob import glob
from astropy.io import fits

In [11]:
bias_files = sorted(glob(os.path.join('/home/srinivas/Documents/observation_data_copy/Test/source_1','bias*')))
print(f"You have {len(bias_files)} bias files")

You have 3 bias files


In [12]:
data = fits.open(bias_files[0])
header = data[0].header
image = data[0].data
nl  = (header['NAXIS1']*header['NAXIS2']*0.1)/len(bias_files)
nhigh = (header['NAXIS1']*header['NAXIS2']*0.1)/len(bias_files)
image

array([[  0,   0, 633, ..., 540, 530, 532],
       [536,   0, 626, ..., 544, 544, 548],
       [525,   0, 640, ..., 555, 539, 536],
       ...,
       [538,   0, 652, ..., 543, 536, 538],
       [540,   0, 656, ..., 539, 551, 541],
       [551,   0, 646, ..., 548, 540, 540]], dtype=uint16)

In [ ]:
for bfiles in bias_files:
    data = fits.open(bfiles)
    header = data[0].header
    image = data[0].data
    nl = header['NAXIS1']*header['NAXIS2']